Test data script. Gathers historical data from the api and inserts it into pagilla. Only collects data up to July 2022 for unknown reasons, resulting in a gap between this table and the update table. Number of currencies also limited due to the time taken for each iteration during testing. The remnant code is present in a comment

In [2]:
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime as dt
from datetime import datetime as d2
from dateutil.relativedelta import relativedelta
import psycopg2
import pandas.io.sql as sqlio

Full test data script

In [ ]:
#Initialise time variables
b = dt.datetime.now()
c = b.strftime("%Y-%m-%d")
d = b - relativedelta(years=3)
e = d.strftime("%Y-%m-%d")

#Generate timestamp list
stamp_url = f'https://api.coindesk.com/v1/bpi/historical/close.json?start={e}&end={c}'
response4 = requests.get(stamp_url)
temp = response4.json()
date_list = list(temp['bpi'])
timestamp = []
for s in date_list:
  timestamp += s.split(',')

#Create dataframe from timestamps
#el df_time
df_time = pd.DataFrame({'date':timestamp})

#Pull data for all languages and insert into dataframe
currency_url = 'https://api.coindesk.com/v1/bpi/supported-currencies.json'
currency_aba = []
time_url = f'https://api.coindesk.com/v1/bpi/historical/close.json?start={e}&end={c}&currency={currency_aba}'
response1 = requests.get(currency_url)
code = response1.json()

#for i,j in enumerate(code):
  #currency_code.append([code[i]['country'], code[i]['currency']][1])
#currency_code = currency_code[:75]
currency_code = ['USD','GBP','JPY','RUB','AED']
for i in currency_code:
  currency_aba = i
  time_url = f'https://api.coindesk.com/v1/bpi/historical/close.json?start={e}&end={c}&currency={currency_aba}'
  response2 = requests.get(time_url)
  text = response2.json()
  n_list = list(text['bpi'])
  m = []
  for s in n_list:
    m += s.split(',')
  rate = []
  for i in m:
    rate.append(text['bpi'][i])
  df_time[f'{currency_aba}'] = rate


#Output dataframe to pagila database for long-term storage
conn = psycopg2.connect(host="data-sandbox.c1tykfvfhpit.eu-west-2.rds.amazonaws.com",
                        database = "pagila",
                        user="de8_dobu28",
                        password="JQmhe99+",
                        port=5432,
                        options='-c search_path=student')

cur = conn.cursor()

cur.execute("CREATE TABLE db_cap_datablock (date timestamp PRIMARY KEY, USD float, GBP float, JPY float, RUB float, AED float);")

query="INSERT INTO db_cap_datablock (date, USD, GBP, JPY, RUB, AED) VALUES (%s, %s, %s, %s, %s, %s)"
record = df_time.to_records(index = False)
cur.executemany(query, record)

conn.commit()

cur.close()
conn.close()